In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# Reading all files

In [ ]:
pd.set_option('max_columns', None)

In [ ]:
cities = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/Cities.csv", encoding = "utf-8")
conference = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/Conferences.csv", encoding = "utf-8")
game_city = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WGameCities.csv", encoding = "utf-8")
NCAATourneyCompactResults = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WNCAATourneyCompactResults.csv", encoding = "utf-8")
NCAATourneyDetailedResults = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WNCAATourneyDetailedResults.csv", encoding = "utf-8")
NCAATourneySeeds = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WNCAATourneySeeds.csv", encoding = "utf-8")
NCAATourneySlots = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WNCAATourneySlots.csv", encoding = "utf-8")
RegularSeasonCompactResults = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WRegularSeasonCompactResults.csv", encoding = "utf-8")
RegularSeasonDetailedResults = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WRegularSeasonDetailedResults.csv", encoding = "utf-8")
sub_stage1 = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WSampleSubmissionStage1.csv", encoding = "utf-8")
seasons = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WSeasons.csv", encoding = "utf-8")
TeamConferences = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WTeamConferences.csv", encoding = "utf-8")
teams = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WTeams.csv", encoding = "utf-8")
TeamSpellings = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WTeamSpellings.csv", encoding = "cp1252")

# Looking all data files one by one

# Cities

In [ ]:
cities.head()

In [ ]:
print("the shape of the data file ---->",cities.shape)
print("Number of cities and states in the data file --->", cities.nunique())

In [ ]:
# plot how many cities in each state
plt.figure(figsize=(20,8))
state_wise = cities.groupby("State")["City"].count().reset_index()
sns.barplot(data=state_wise, x="State", y="City")


# Conference

**ConfAbbrev** -- small abbrevation of the conference 
**Description** -- Conference name

In [ ]:
conference.head()

In [ ]:
print("Number of conferneces happended based on conference description--->", conference["Description"].nunique())
print("Number of conferneces happended based on conference abbrevation--->", conference["ConfAbbrev"].nunique())

# Game city

**Season** - this is the year of the associated entry in WSeasons.csv (the year in which the final tournament occurs). For example, during the 2016 season, there were regular season games played between November 2015 and March 2016, and all of those games will show up with a Season of 2016. <br>
**DayNum** - this integer always ranges from 0 to 132, and tells you what day the game was played on. It represents an offset from the "DayZero" date in the "WSeasons.csv" file. For example, the first game in the file was DayNum=18. Combined with the fact from the "WSeasons.csv" file that day zero was 10/27/1997 that year, this means the first game was played 18 days later, or 11/14/1997. There are no teams that ever played more than one game on a given date, so you can use this fact if you need a unique key (combining Season and DayNum and WTeamID).<br>
**WTeamID** - this identifies the id number of the team that won the game, as listed in the "WTeams.csv" file. No matter whether the game was won by the home team or visiting team, or if it was a neutral-site game, the "WTeamID" always identifies the winning team.<br>

**LTeamID** - this identifies the id number of the team that lost the game.<br>
**CRType** - this can be either Regular or NCAA or Secondary. If it is Regular, you can find more about the game in the MRegularSeasonCompactResults.csv and MRegularSeasonDetailedResults.csv files. If it is NCAA, you can find more about the game in the MNCAATourneyCompactResults.csv and MNCAATourneyDetailedResults.csv files. If it is Secondary, you can find more about the game in the MSecondaryTourneyCompactResults file.<br>
**CityID** - the ID of the city where the game was played, as specified by the CityID column in the Cities.csv file.

In [ ]:
game_city.head()

In [ ]:
# count the CRType feature
sns.countplot(data=game_city, y="CRType")

In [ ]:
# plot the season count
display(pd.DataFrame(game_city["Season"].value_counts()).T)
sns.countplot(data=game_city, x="Season")

In [ ]:
# which team wins more
print("Number of winning teams",game_city["WTeamID"].nunique())
sns.countplot(data=game_city, x="WTeamID")

# Seasons

This file identifies the different seasons included in the historical data, along with certain season-level properties.

**Season** - indicates the year in which the tournament was played. Remember that the current season counts as 2022.<br>
**DayZero** - tells you the date corresponding to DayNum=0 during that season. All game dates have been aligned upon a common scale so that (each year) Selection Monday is on day 133. All game data includes the day number in order to make it easier to perform date calculations. If you need to know the exact date a game was played on, you can combine the game's "DayNum" with the season's "DayZero". For instance, since day zero during the 2011-2012 season was 10/31/2011, if we know that the earliest regular season games that year were played on DayNum=7, they were therefore played on 11/07/2011.<br>
**RegionW, RegionX, Region Y, Region Z** - by convention, the four regions in the final tournament are always named W, X, Y, and Z. Whichever region's name comes first alphabetically, that region will be Region W. And whichever Region plays against Region W in the national semifinals, that will be Region X. For the other two regions, whichever region's name comes first alphabetically, that region will be Region Y, and the other will be Region Z. This allows us to identify the regions and brackets in a standardized way in other files. For instance, during the 2012 tournament, the four regions were DesMoines, Fresno, Kingston, and Raleigh. Being the first alphabetically, DesMoines becomes W. Since the Fresno regional champion (Stanford) played against the DesMoines regional champion (Baylor) in the national semifinals, that makes Fresno be region X. For the other two (Kingston and Raleigh), since Kingston comes first alphabetically, that makes Kingston Y and therefore Raleigh is Z. So for that season, the W/X/Y/Z are DesMoines,Fresno,Kingston,Raleigh. And so for instance, Baylor, the #1 seed in the DesMoines region, is listed in the WNCAATourneySeeds file with a seed of W01, meaning they were the #1 seed in the W region (the DesMoines region). We will not know the final W/X/Y/Z designations until Selection Monday, because the national semifinal pairings in the Final Four will depend upon the overall ranks of the four #1 seeds.
<br><br>
The game dates in this dataset are expressed in relative terms, as the number of days since the start of the regular season, and aligned for each season so that day number #133 is the Monday right before the tournament, when team selections are made. During any given season, day number zero is defined to be exactly 19 weeks earlier than Selection Monday, so Day #0 is a Monday in late October or early November such that Day #132 is Selection Sunday (for the men's tournament) and Day #133 is Selection Monday (for the women's tournament).
<br><br>
This doesn't necessarily mean that the regular season will always start exactly on day #0 or day #1; in fact, during the past decade, regular season games typically start being played on a Friday that is either Day #4 or Day #11, but further back there was more variety.



In [ ]:
print("Shape of the data file", seasons.shape)
seasons.head(25)


In [ ]:
# see how many regions are there
regions = []
regi_col = ["RegionW", "RegionX", "RegionY", "RegionZ"]
for r in regi_col:
    for region_name in seasons.loc[:,r].values:
        regions.append(region_name)
print("How many unique regions are partipating in this game from 1998 - 2022 ---->", pd.Series(regions).nunique())

In [ ]:
# what are the region names
set(regions)

In [ ]:
# RegionW -- plot
fig = plt.figure(figsize=(20, 4))
sns.countplot(data=seasons, x='RegionW');
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 4))
sns.countplot(data=seasons, x='RegionY');
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 4))
sns.countplot(data=seasons, x='RegionX');
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 4))
sns.countplot(data=seasons, x='RegionZ');
plt.show()

# Team Conferences

In [ ]:

print("Shape of the data file --->",TeamConferences.shape)
TeamConferences.head()

In [ ]:
print("How many team conferences were happend -->",TeamConferences["ConfAbbrev"].nunique())

In [ ]:
# plot the count of team conferences according to a year
display(pd.DataFrame(TeamConferences.Season.value_counts()).T)
plt.figure(figsize=(20,5))
plt.title("How many conferences are happening per year")
sns.countplot(data=TeamConferences, x="Season")

# Teams

**TeamID** - a 4 digit id number, from 1000-1999, uniquely identifying each NCAA® men's team. A school's TeamID does not change from one year to the next, so for instance the Duke men's TeamID is 1181 for all seasons. To avoid possible confusion between the men's data and the women's data, all of the men's team ID's range from 1000-1999, whereas all of the women's team ID's range from 3000-3999.<br>
**TeamName** - a compact spelling of the team's college name, 16 characters or fewer. There are no commas or double-quotes in the team names, but you will see some characters that are not letters or spaces, e.g., Texas A&M, St Mary's CA, TAM C. Christi, and Bethune-Cookman.

In [ ]:
print("Number of teams in the compatition --->",teams.shape[0])
teams.head()

# Team Spelling file

This file indicates alternative spellings of many team names. It is intended for use in associating external spellings against our own TeamID numbers, thereby helping to relate the external data properly with our datasets. Over the years we have identified various external spellings of different team names (as an example, for Ball State we have seen "ball st", and "ball st.", and "ball state", and "ball-st", and "ball-state"). Other teams have had more significant changes to their names over the years; for example, "Texas Pan-American" and "Texas-Rio Grande Valley" are actually the same school. The current list is obviously not exhaustive, and we encourage participants to identify additional mappings and upload extended versions of this file to the forums.

**TeamNameSpelling** - this is the spelling of the team name. It is always expressed in all lowercase letters - e.g. "ball state" rather than "Ball State" - in order to emphasize that any comparisons should be case-insensitive when matching.<br>
**TeamID** - this identifies the TeamID for the team that has the alternative spelling (as described in MTeams.csv).

In [ ]:
print("Shape of the data file --->",TeamSpellings.shape)
TeamSpellings.head()

In [ ]:
print("How many teams are there ---->",TeamSpellings.TeamID.nunique())

In [ ]:
# How many different spellings are there for a team
plt.figure(figsize=(30,5))
sns.countplot(data=TeamSpellings, x="TeamID")

# NCCA Tournament Seeds

This file identifies the seeds for all teams in each NCAA® tournament, for all seasons of historical data. Thus, there are between 64-68 rows for each year, depending on whether there were any play-in games and how many there were. In recent years the structure has settled at 68 total teams, with four "play-in" games leading to the final field of 64 teams entering Round 1 on Thursday of the first week (by definition, that is DayNum=136 each season). We will not know the seeds of the respective tournament teams, or even exactly which 68 teams it will be, until Selection Sunday on March 13, 2022 (DayNum=132).

**Season** - the year that the tournament was played in <br>
**Seed** - this is a 3/4-character identifier of the seed, where the first character is either W, X, Y, or Z (identifying the region the team was in) and the next two digits (either 01, 02, ..., 15, or 16) tell you the seed within the region. For play-in teams, there is a fourth character (a or b) to further distinguish the seeds, since teams that face each other in the play-in games will have seeds with the same first three characters. The "a" and "b" are assigned based on which Team ID is lower numerically. As an example of the format of the seed, the first record in the file is seed W01 from 1985, which means we are looking at the #1 seed in the W region (which we can see from the "MSeasons.csv" file was the East region).<br>
TeamID - this identifies the id number of the team, as specified in the MTeams.csv file

In [ ]:
print("Shape of the data file --->",NCAATourneySeeds.shape)
NCAATourneySeeds.head()

In [ ]:
print("Total Seasons -- >",(NCAATourneySeeds.Season.nunique()))
print("Seeds Count -->",(NCAATourneySeeds.Seed.nunique()))
print("Unique TeamID -->",(NCAATourneySeeds.TeamID.nunique()))

In [ ]:
NCAATourneySeeds.Seed.value_counts() # which means One seed for every year - like seed W01 will be there for all 23 years

# NCCA Tournament Slots

This file identifies the mechanism by which teams are paired against each other, depending upon their seeds, as the tournament proceeds through its rounds. It can be of use in identifying, for a given historical game, what round it occurred in, and what the seeds/slots were for the two teams (the meaning of "slots" is described below). Because of the existence of play-in games for particular seed numbers, the pairings have small differences from year to year. You may need to know these specifics if you are trying to represent/simulate the exact workings of the tournament bracket.

**Season** - this is the year of the associated entry in MSeasons.csv (the year in which the final tournament occurs)
Slot - this uniquely identifies one of the tournament games. For play-in games, it is a three-character string identifying the seed fulfilled by the winning team, such as W16 or Z13. For regular tournament games, it is a four-character string, where the first two characters tell you which round the game is (R1, R2, R3, R4, R5, or R6) and the second two characters tell you the expected seed of the favored team. Thus the first row is R1W1, identifying the Round 1 game played in the W bracket, where the favored team is the 1 seed. As a further example, the R2W1 slot indicates the Round 2 game that would have the 1 seed from the W bracket, assuming that all favored teams have won up to that point. Even if that R2W1 slot were actually a game between the W09 and W16 teams, it is still considered to be the R2W1 slot. The slot names are different for the final two rounds, where R5WX identifies the national semifinal game between the winners of regions W and X, and R5YZ identifies the national semifinal game between the winners of regions Y and Z, and R6CH identifies the championship game. The "slot" value is used in other columns in order to represent the advancement and pairings of winners of previous games.<br><br>
**StrongSeed** - this indicates the expected stronger-seeded team that plays in this game. For Round 1 games, a team seed is identified in this column (as listed in the "Seed" column in the MNCAATourneySeeds.csv file), whereas for subsequent games, a slot is identified in this column. In the first record of this file (slot R1W1), we see that seed W01 is the "StrongSeed", which during the 1985 tournament would have been Georgetown. Whereas for games from Round 2 or later, rather than a team seed, we will see a "slot" referenced in this column. So in the 33rd record of this file (slot R2W1), it tells us that the winners of slots R1W1 and R1W8 will face each other in Round 2. Of course, in the last few games of the tournament - the national semifinals and finals - it's not really meaningful to talk about a "strong seed" or "weak seed", since you would have #1 seeds favored to face each other, but those games are nevertheless represented in the same format for the sake of consistency.<br><br>
**WeakSeed** - this indicates the expected weaker-seeded team that plays in this game, assuming all favored teams have won so far. For Round 1 games, a team seed is identified in this column (as listed in the "Seed" column in the MNCAATourneySeeds.csv file), whereas for subsequent games, a slot is identified in this column.<br><br>

In [ ]:
print("Shape of the data file --->",NCAATourneySlots.shape)
NCAATourneySlots.head()

# NCAA Tournament Compact Results

This file identifies the game-by-game NCAA® tournament results for all seasons of historical data. The data is formatted exactly like the WRegularSeasonCompactResults data. Each season you will see 63 games listed, since there are no women's play-in games.

Although the scheduling of the men's tournament rounds has been consistent for many years, there has been more variety in the scheduling of the women's rounds. 

**Season** - this is the year of the associated entry in WSeasons.csv (the year in which the final tournament occurs). For example, during the 2016 season, there were regular season games played between November 2015 and March 2016, and all of those games will show up with a Season of 2016.<br>
**DayNum** - this integer always ranges from 0 to 132, and tells you what day the game was played on. It represents an offset from the "DayZero" date in the "WSeasons.csv" file. For example, the first game in the file was DayNum=18. Combined with the fact from the "WSeasons.csv" file that day zero was 10/27/1997 that year, this means the first game was played 18 days later, or 11/14/1997. There are no teams that ever played more than one game on a given date, so you can use this fact if you need a unique key (combining Season and DayNum and WTeamID).<br>
**WTeamID** - this identifies the id number of the team that won the game, as listed in the "WTeams.csv" file. No matter whether the game was won by the home team or visiting team, or if it was a neutral-site game, the "WTeamID" always identifies the winning team.<br>
**WScore** - this identifies the number of points scored by the winning team.<br>
**LTeamID** - this identifies the id number of the team that lost the game.<br>
**LScore** - this identifies the number of points scored by the losing team. Thus you can be confident that WScore will be greater than LScore for all games listed.<br>
**NumOT** - this indicates the number of overtime periods in the game, an integer 0 or higher.<br>
**WLoc** - this identifies the "location" of the winning team. If the winning team was the home team, this value will be "H". If the winning team was the visiting team, this value will be "A". If it was played on a neutral court, then this value will be "N". Sometimes it is unclear whether the site should be considered neutral, since it is near one team's home court, or even on their court during a tournament, but for this determination we have simply used the Kenneth Massey data in its current state, where the "@" sign is either listed with the winning team, the losing team, or neither team. If you would like to investigate this factor more closely, we invite you to explore Data Section 3, which provides the city that each game was played in, irrespective of whether it was considered to be a neutral site.

In [ ]:
print("shape of the data --->",NCAATourneyCompactResults.shape)
NCAATourneyCompactResults.head()

In [ ]:
NCAATourneyCompactResults["points_diff"] = NCAATourneyCompactResults.WScore - NCAATourneyCompactResults.LScore
# plot the point difference
plt.figure(figsize=(30,5))
sns.countplot(data=NCAATourneyCompactResults, x="points_diff")

In [ ]:
plt.figure(figsize=(30,5))
sns.histplot(data=NCAATourneyCompactResults, x="points_diff", bins=40)

In [ ]:
summaries = NCAATourneyCompactResults[['Season', 
    'WScore', 
    'LScore', 
    'NumOT', 
    'points_diff']].groupby('Season').agg(['min', 'max', 'mean', 'median'])
summaries.columns = ["-".join(col) for col in summaries.columns.values]
summaries

In [ ]:
summaries[[col for col in summaries.columns if 'points_diff' in col]].plot(figsize=(12,8))
plt.title('Point difference over seasons')
plt.show()

# NCAA Tournament Detailed results

In [ ]:
print("Shape of the data --->",NCAATourneyDetailedResults.shape)
NCAATourneyDetailedResults.head()

# Submission

In [ ]:
sub_stage1.head()

# Generating training data

Loading ....